# ARIM-Academy：　基礎編　Scikit-learn（分類モデル）

## 本編の目標

本演習では**『茶の元素分析データセット』**を用いて教師なし機械学習の分類技術を学びます。このデータセットには、4種類の茶葉（ブラックセイロン、ブラックトルコ、グリーンセイロン、グリーントルコ）の元素分析データが含まれています。

### 本編における内容
以下の内容に取り組みます。

* **データの可視化と解釈**: 散布図やペアプロットを用いて、特徴量の分布や相関関係を視覚的に把握します。
  
* **分類アルゴリズムの実装**: 決定木、ランダムフォレスト、サポートベクターマシーンらの分類アルゴリズムを実装し、それぞれの特徴を理解します。
  
* **モデル評価** : 混同行列を用いてモデルの予測結果と正解ラベルを比較し、性能を評価する方法を習得します。  
---

## データセット

**『茶の元素分析データセット』**は、ブラックセイロン(BC)、ブラックトルコ(BT)、グリーンセイロン(GC)、グリーントルコ(GT)の4種類の茶葉について、3つの濃度（1%、2%、3%）で抽出した液体の元素分析データです。誘導結合プラズマ発光分光分析法（ICP-OES）で分析された元素は、Al, Ca, Cd, Cr, Cu, Hg, Fe, K, Mg, Mn, Na, Pb, Znです。[1]

[1] Durmus, Y., Atasoy, A.D. & Atasoy, A.F. Mathematical optimization of multilinear and artificial neural network regressions for mineral composition of different tea types infusions. Sci Rep 14, 18285 (2024). https://doi.org/10.1038/s41598-024-69149-1

### 教材への接続
google colabにおけるオンラインの場合にこのラインを実行します。（<font color="red">Google colabに接続しない場合には不要</font>）

In [ ]:
!git clone https://github.com/ARIM-Academy/Advanced_Tutorial_1.git
%cd Advanced_Tutorial_1

# 1.データセットの読み込みと前処理

### ライブラリのインポート
カリキュラムで扱うpythonのライブラリを`import`文でロードします。機械学習のライブラリであるscikit-learnは後半でimportします。

In [ ]:
#ライブラリ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### サンプルファイルの読み込み
pandasライブラリの`read_csv()`はcsvファイルを読み込むメソッドであり、指定したファイルの読み込みます。ここでは[data]フォルダーに格納されているIris.csvのファイルのをデータフレームとして読み込み、そのデータフレームはdfという変数に格納します。

In [ ]:
#データセットの読み込み
df=pd.read_csv("data/tea_data.csv")

In [ ]:
#データセットの確認
df

# 2. データの俯瞰（探索的データ分析）

### ①概要統計
元素分析で観測された９つの元素成分の概要統計を出力します。

In [ ]:
#概要統計
df.describe()

### ②　ペアプロット（散布図行列）

茶の元素分析データセットにペアプロットを適用することで、以下の情報を得ることができます。

* **特徴量間の関係**: 異なる特徴量間の関係を視覚的に確認できます。例えば、AlとZnが正の相関関係にあるか、負の相関関係にあるかなどを一目でわかります。
* **クラス間の分離性**: 各品種（ブラックセイロン(BC)、ブラックトルコ(BT)、グリーンセイロン(GC)、グリーントルコ(GT)）のデータがどのように分布しているか、他の品種とどの程度分離しているかを確認できます。
* **外れ値の検出**: データに含まれる外れ値を視覚的に確認することができます。
* **特徴量選択**: どの特徴量が品種の分類に最も貢献しているか、あるいは冗長な特徴量があるかなどを判断する手がかりになります。

In [ ]:
#　ペアプロットによる各変数間の二次元空間像
sns.pairplot(df)
plt.show()

In [ ]:
#　ペアプロットによる各変数間の二次元空間像（その２）
sns.pairplot(df,hue='tea')
plt.show()

In [ ]:
#　ペアプロットによる各変数間の二次元空間像（その３）
sns.pairplot(df,hue='tea',diag_kind='hist')
plt.show()

# 3. 機械学習モデルの比較

### データセットの分割（Data set splitting）
データセットの分割（Data set splitting）は、機械学習モデルの訓練と評価のために、データセットを訓練データとテストデータに分けるプロセスを指します。モデルがテストデータに対してどれくらい正確に予測できるかを評価することで、モデルの汎化能力や性能を測定します。

**訓練データ（Training Data）**：　機械学習モデルの訓練に使用されるデータセットです。モデルは、訓練データを分析してパターンや関連性を学習し、未知のデータに対して予測を行うための知識を獲得します。

**テストデータ（Test Data）**：モデルの訓練後に性能を評価するために使用されるデータセットです。テストデータは、訓練データとは異なるデータでなければなりません。

In [ ]:
# 説明変数と目的変数の用意しましょう
X = df.iloc[0:, 0:9] #　元素分析値
y = df.iloc[0:, 9]   # 茶の品種

In [ ]:
X

In [ ]:
y

### 標準化
モデルはデータのスケールに敏感であるため、次元削減の前に標準化が必要です。特に線形回帰やサポートベクターマシン（SVM）などのアルゴリズムで重要です。これらのモデルは、特徴量のスケールが異なると結果に大きく影響を受けるため、データの標準化は不可欠な前処理のステップとなります。

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# データの標準化
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

### データセットの分割（Data set splitting）
機械学習モデルの性能を正確に評価するためには、モデルを学習させる**訓練データ**と、学習済みのモデルの性能を評価するための**テストデータ**にデータを分割する必要があります。scikit-learnのtrain_test_split()関数を使用すると、この分割を簡単に実行できます。

In [ ]:
#データセットの分割のライブラリの読み込み
from sklearn.model_selection import train_test_split

# train_test_split関数によるデータセット分割
X_train, X_test, y_train, y_test = train_test_split(X_std, y,
                                                    test_size = 0.2,   # 20%をテストデータとするデータ分割
                                                    stratify = y,      # ４つの品種について均等分割
                                                    random_state=42
                                                    )

In [ ]:
X_train

In [ ]:
y_train

### scikit-learnによる分類モデルの選択

scikit-learnは、機械学習のアルゴリズムをPythonで実装するためのライブラリです。特に、分類問題においては、ロジスティック回帰、k近傍法、サポートベクターマシン、決定木、ランダムフォレスト、ニューラルネットワークなど、様々なアルゴリズムが提供されています。どのモデルを選ぶべきかは、**データの特性**（線形か非線形か、データの量、ノイズの有無など）や、**タスクの要件**（精度、解釈性、計算時間など）によって異なります。

**主な分類モデルの特徴**

* **ロジスティック回帰:**
  - 線形モデルの一種で、入力データからクラスの所属確率を予測します。
  - データが線形に分離可能な場合や、特徴量の重要度を把握したい場合に適しています。
* **k近傍法:**
  - 新しいデータ点に対して、最も近いk個のデータ点のラベルを参照して、そのデータ点のラベルを決定します。
  - データが局所的な構造を持つ場合や、非線形な境界を持つ場合に有効です。
* **サポートベクターマシン:**
  - データを最もよく分離する超平面（決定境界）を見つけ、新しいデータを分類します。
  - マージン最大化という概念に基づいており、一般化性能が高いことが期待できます。
* **決定木:**
  - データを順次分割していくことで、決定規則を樹形図で表現するモデルです。
  - 非線形な関係を捉えることができ、解釈性が高い点が特徴です。
* **ランダムフォレスト:**
  - 複数の決定木を組み合わせることで、過学習を防ぎ、汎化性能を向上させたモデルです。
  - さまざまな種類のデータに対して高い精度を発揮します。
* **ニューラルネットワーク:**
  - 人間の脳神経回路を模倣したモデルで、複雑なパターンを学習することができます。
  - 大規模なデータセットや画像認識、自然言語処理など、高度なタスクに適用されます。

**モデル選択のポイント**

* **データの探索:** データの分布、特徴量間の関係、外れ値などを可視化し、データの特性を把握します。
* **アルゴリズムの特性:** 各アルゴリズムの強みと弱みを理解し、データの特性に合ったアルゴリズムを選びます。
* **ハイパーパラメータチューニング:** アルゴリズムのパラメータを調整することで、モデルの性能を向上させることができます。
* **交差検証:** モデルの汎化性能を評価するために、交差検証を行います。

scikit-learnは、様々な分類モデルを提供しており、適切なモデルを選択することで、高い精度で分類問題を解決することができます。モデル選択は、データの特性やタスクの要件に合わせて、慎重に行う必要があります。ここでは決定木、ランダムフォレスト、サポートベクターマシーンの3つについてのモデルを検討します。

## 1. 決定木
決定木は、データの属性値を基に、一連のif-thenルールを適用することで、目的変数を予測する非線形モデルです。

### ①　モデル構築
DecisionTreeClassifier クラスは、Scikit-learnライブラリが提供するクラスで、分類型による決定木モデルを構築するために使用されます。

In [ ]:
# 決定木法のインポート（クラス指定）
from sklearn.tree import DecisionTreeClassifier

# 決定木法によるモデル構築（トレーニング）
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

#### ②　モデルの評価：精度の計算
#### 訓練データ

In [ ]:
#訓練データの予測ラベル
y_pred_train = model.predict(X_train)

In [ ]:
# 訓練データの精度
r2_score = model.score(X_train, y_train)
print('訓練データ決定係数:{:.3f}'.format(r2_score))

#### テストデータ

In [ ]:
#テストデータの予測ラベル
y_pred_test = model.predict(X_test)

In [ ]:
# テストデータの精度
r2_score = model.score(X_test, y_test)
print('テストデータ決定係数:{:.3f}'.format(r2_score))

### ③　混同行列

混同行列は、機械学習の分類問題において、モデルの性能を評価するために用いられる指標です。特に、不均衡データ（あるクラスのデータが圧倒的に多いデータセット）や、コスト感度の高い誤分類（ある種類の誤分類が他の種類よりも重大な場合）を扱う際に、これらの指標は非常に有効です。  

混同行列:予測結果と実際のラベルを2×2の表にまとめたもので、表の各要素は、以下の4つの状態を表します。  
* TP (True Positive): 正しく陽性と予測  
* FP (False Positive): 誤って陽性と予測（偽陽性）  
* TN (True Negative): 正しく陰性と予測  
* FN (False Negative): 誤って陰性と予測（偽陰性）  

混同行列から、精度、適合率、再現率などの評価指標を計算できます。  

#### 訓練データ

**混同行列の算出　その１**  
sklearn.metrics モジュールにある混同行列を計算するための関数 confusion_matrix を用いる方法です。なお、sklearn.metrics はscikit-learnライブラリが提供する様々な評価指標を計算するための関数群です。

In [ ]:
# scikit-learnのconfution_matrixを使った混同行列の表記方法

from sklearn import metrics
print(metrics.confusion_matrix(y_pred_train, y_train))

**混同行列の算出　その2**  
Pandasライブラリのcrosstab関数を使用して、混同行列を作成します。

In [ ]:
# pandasのcrosstbを使った混同行列の別の表記方法

pd.crosstab(y_pred_train, y_train, margins = True)

#### テストデータ

In [ ]:
# pandasのcrosstbを使った混同行列の別の表記方法

pd.crosstab(y_pred_test, y_test, margins = True)

In [ ]:
print(metrics.confusion_matrix(y_pred_test, y_test))

## 2. ランダムフォレスト
RandomForestClassifier クラスは、Scikit-learnライブラリが提供するクラスで、分類型によるランダムフォレストモデルを構築するために使用されます。

#### ①モデル構築

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

### ②　モデルの評価：精度の計算および混同行列
#### 訓練データ

In [ ]:
y_pred_train = model.predict(X_train)

In [ ]:
print ("RandomForestClassifier training score : {:.3f}".format(model.score(X_train, y_train)))
pd.crosstab(y_pred_train, y_train, margins = True)

#### テストデータ

In [ ]:
y_pred_test = model.predict(X_test)

In [ ]:
print ("RandomForestClassifier test score : {:.3f}".format(model.score(X_test, y_test)))
pd.crosstab(y_pred_test, y_test, margins = True)

### 3.サポートベクターマシーン

SVC クラスは、Scikit-learnライブラリが提供するクラスで、分類型によるサポートベクターマシーンモデルを構築するために使用されます。

### ① モデル構築

In [ ]:
# サポートベクター
from sklearn.svm import SVC

model = SVC()
model.fit(X_train, y_train)

### ②　モデルの評価：精度の計算および混同行列
#### 訓練データ

In [ ]:
y_pred_train = model.predict(X_train)

In [ ]:
print ("SupportVectorClassifier training score : {:,.3f}".format(model.score(X_train, y_train)))
pd.crosstab(y_pred_train, y_train, margins = True)

#### テストデータ

In [ ]:
y_pred_test = model.predict(X_test)

In [ ]:
print ("SupportVectorClassifier test score : {:,.3f}".format(model.score(X_test, y_test)))
pd.crosstab(y_pred_test, y_test, margins = True)